In [1]:
import os
from openai import OpenAI
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    documents_raw = json.load(f_in)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [3]:
index = minsearch.Index(
        text_fields=['section', 'question', 'text'],
        keyword_fields=['course']
)

q = "the course already started, can i still join"
index.fit(documents)

boost = {'question': 3.0, 'text': 0.5}
filter_dict = {'course': 'data-engineering-zoomcamp'}

results = index.search(
    query=q,
    filter_dict=filter_dict,
    boost_dict=boost,
    num_results=5
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [4]:
client = OpenAI(api_key=os.environ.get("OPEN_API_KEY"),
)

In [5]:
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=[{"role":"user", "content": q}]
)

response.choices[0].message.content

"Typically, joining a course after it has already started depends on several factors, including the rules and flexibility of the institution, the policies of the specific course, and how much of the course content you have missed. Here are some steps you could take:\n\n1. **Contact the Instructor:** Reach out to the course instructor or professor and explain your situation. They may allow you to join late, especially if you are willing to catch up on missed work.\n\n2. **Check with the Administration:** Contact the academic office or department offering the course. They can provide information on late registration policies and any potential fees or paperwork that may be required.\n\n3. **Assess Feasibility:** Consider how much coursework you have missed and whether you will realistically be able to catch up without negatively affecting your performance in the course.\n\n4. **Review the Syllabus:** If you get permission to join, review the syllabus thoroughly to understand the course st

In [6]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Answer the QUESTION using only the facts from the CONTEXT. 
If you cannot answer the QUESTION using the context, return NONE.


QUESTION: {question}

CONTEXT: {context}

""".strip()


context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [7]:
q

'the course already started, can i still join'

In [8]:
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=[{"role":"user", "content": prompt_template.strip().format(question=q, context=context.strip())}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [9]:
def search(query):
    boost = {'question': 3.0, 'text': 0.5}
    filter_dict = {'course': 'data-engineering-zoomcamp'}
    
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=11
    )
    
    return results

def get_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ table.
    Answer the QUESTION only using facts from the CONTEXT. 
    If you cannot answer the QUESTION using the CONTEXT, return NONE.
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()


    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return prompt_template.format(question=query, context=context.strip()).strip()


def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages=[{"role":"user", "content": prompt}]
    )

    return response.choices[0].message.content


def rag(q):
    search_results = search(q)
    # print(search_results)
    prompt = get_prompt(q, search_results)
    # print(prompt)
    answer = llm(prompt)
    print()
    print()
    return answer

q1 = 'the course already started, can i still join?'
q2 = 'how do i run kafka?'
q3 = 'how do i run dbt?'
q4 = 'I missed the live course. Can i still get a certificate?'
# search(q2)
q = q2

# print(get_prompt(q, search(q)))
print()
print(rag(q))




In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java


# using Elastic search

In [10]:
from elasticsearch import Elasticsearch


es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
from tqdm.auto import tqdm


/Users/talldarkandhandsome/Documents/development/llm-zoomcamp/llm-zoomcamp-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████| 948/948 [00:01<00:00, 474.71it/s]


In [17]:
query = 'I just disovered the course. Can I still join it?'

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text^0.5", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs



def rag(query):
    search_results = elastic_search(query)
    prompt = get_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
rag(query)


'Yes, you can still join the course even after the start date. You are eligible to submit the homeworks, but be aware of the deadlines for the final projects.'

# Homework

## Q3

In [24]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    scores = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        scores.append(hit['_score'])
    
    return result_docs, scores


In [25]:
q3_ans = elastic_search(query)
q3_ans

([{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (

## Q4

In [27]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    scores = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        scores.append(hit['_score'])
    
    return result_docs[:3], scores[:3]


q4_ans = elastic_search(query)
q4_ans

([{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker contain

## Q5

In [36]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for res in q4_ans[0]:
    context += context_template.format(question=res['question'], text=res['text']) + "\n\n"

# print(context)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()



prompt = prompt_template.format(
            question="How do I execute a command in a running docker container?",
            context=context
).strip()


print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [38]:
len(prompt)

1462

## Q6

In [39]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")


In [40]:
len(encoding.encode(prompt))

322